In [ ]:
import numpy as np
from gurobipy import Model, GRB, quicksum
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf
from tensorflow.keras.models import load_model

np.random.seed(42)

# número de objetos por mochila
n = 5  
# número de escenarios a generar
num_escenarios = 10000

datos_entrada = []
etiquetas_salida = []

# resolver el problema de la mochila para cada escenario
def resolver_problema_mochila_binaria(n, W, weights, values):
    model = Model()
    x = model.addVars(n, vtype=GRB.BINARY, name="x")
    model.setObjective(quicksum(values[i] * x[i] for i in range(n)), GRB.MAXIMIZE)
    model.addConstr(quicksum(weights[i] * x[i] for i in range(n)) <= W, "WeightLimit")
    model.setParam("OutputFlag", 0)
    model.optimize()
    return [1 if x[i].x > 0.5 else 0 for i in range(n)]

# generación de datos
for _ in range(num_escenarios):
    W = np.random.randint(5, 20)  # capacidad de la mochila
    weights = np.random.randint(1, 10, n)  # pesos de los objetos
    values = np.random.randint(1, 20, n)  # valores de los objetos
    
    # resolver para obtener la selección óptima
    seleccion_optima = resolver_problema_mochila_binaria(n, W, weights, values)
    
    # entrada: capacidad, pesos y valores
    datos_entrada.append(np.concatenate(([W], weights, values)))

    # salida: capacidad, pesos, y selección óptima
    etiquetas_salida.append(np.concatenate(([W], weights, seleccion_optima)))

datos_entrada = np.array(datos_entrada)
etiquetas_salida = np.array(etiquetas_salida)

# dejar los ultimos 2000 escenarios para test
datos_test = datos_entrada[-2000:]
etiquetas_test = etiquetas_salida[-2000:]

# tomar los primeros 8000 escenarios como datos de entrenamiento
datos_entrada = datos_entrada[:8000]
etiquetas_salida= etiquetas_salida[:8000]

# comprobación de formas
print(f"Forma de datos_entrada: {datos_entrada.shape}")
print(f"Forma de etiquetas_salida: {etiquetas_salida.shape}")


Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-17
Forma de datos_entrada: (8000, 11)
Forma de etiquetas_salida: (8000, 11)


In [ ]:
# definir el modelo de red neuronal
modelo = Sequential([
    Dense(64, activation='relu', input_dim=2*n+1),  # entrada: capacidad, pesos, valores
    Dense(64, activation='relu'),
    Dense(n, activation='sigmoid')  # salida binaria para cada objeto
])


c:\Users\jjacq\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
def loss_mochila(y_true, y_pred):
    # extraer capacidad, pesos, y etiquetas reales desde y_true
    capacidad = y_true[:, 0]
    pesos = y_true[:, 1:n+1]
    y_true_bin = y_true[:, n+1:]

    y_pred_bin = tf.cast(y_pred > 0.5, tf.float32)

    # calcular el peso total de los objetos seleccionados
    peso_total = tf.reduce_sum(y_pred_bin * pesos, axis=1)

    # penalización si se excede la capacidad
    penalizacion = tf.maximum(0.0, peso_total - capacidad)

    bce = tf.keras.losses.binary_crossentropy(y_true_bin, y_pred)

    # combinar BCE con penalización
    return bce + 100 * penalizacion  # factor de penalización arbitrario, pero que indique que es importante no exceder la capacidad


In [ ]:
modelo.compile(optimizer='adam', loss=loss_mochila, metrics=['accuracy'])
modelo.fit(datos_entrada, etiquetas_salida, epochs=50, batch_size=4, verbose=1)


Epoch 1/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.2172 - loss: 174.7225
Epoch 2/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.2087 - loss: 91.4862
Epoch 3/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.2114 - loss: 83.2560
Epoch 4/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.2256 - loss: 72.8606
Epoch 5/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.2118 - loss: 73.9838
Epoch 6/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.2114 - loss: 69.6035
Epoch 7/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.2170 - loss: 64.6853
Epoch 8/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.2245 - loss: 67.8687
Epoch 9/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.2053 - loss: 65.7576
Epoch 10/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.2177 - loss: 64.2080
Epoch 11/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.2131 - loss: 64.3985
Epoch 12/50
2000/2

In [ ]:
loss, accuracy = modelo.evaluate(datos_test, etiquetas_test)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2460 - loss: 59.7647  


In [ ]:
#modelo.save('modelo_mochila.keras') # descomentar para guardar el modelo, pero ya existe uno entrenado

In [ ]:
from tensorflow.keras.models import load_model
modelo = load_model('modelo_mochila.keras', custom_objects={'loss_mochila': loss_mochila})

In [ ]:
def es_factible(seleccion, weights, W):
    return sum(weights[i] for i in range(len(seleccion)) if seleccion[i] == 1) <= W

# función heurística para ajustar la selección si no es factible
def factibilizar_seleccion(weights, seleccion, W):
    peso_total = sum(weights[i] for i in range(len(seleccion)) if seleccion[i] == 1)
    n = len(weights)

    # si la selección no es factible, ajustamos
    if not es_factible(seleccion, weights, W):
        # crear una lista de índices de objetos seleccionados
        indices_seleccionados = [i for i in range(n) if seleccion[i] == 1]
        
        # ordenar objetos seleccionados por su peso
        indices_seleccionados.sort(key=lambda i: weights[i], reverse=True)
        
        # des-seleccionar objetos hasta que la selección sea factible
        for i in indices_seleccionados:
            if peso_total > W:
                seleccion[i] = 0  # deseleccionamos el objeto
                peso_total -= weights[i]  # actualizar el peso total

    return seleccion


In [ ]:
num_factibles = 0
numero_optimos = 0
numero_optimos_post_factibilizacion = 0

for i in range(2000):  # iterar sobre todos los escenarios de test
    entrada = datos_test[i].reshape(1, -1)  # escenario actual
    prediccion = modelo.predict(entrada, verbose=0)  # predicción de la red neuronal
    seleccion = np.round(prediccion[0])  # convertir a selección binaria (0 o 1)
    
    W = entrada[0, 0]  # capacidad
    weights = entrada[0, 1:n+1]  # pesos de los objetos
    
    # verificar factibilidad
    if es_factible(seleccion, weights, W):
        num_factibles += 1
        # verificar si la selección es óptima
        seleccion_optima = etiquetas_test[i][n+1:]
        if np.array_equal(seleccion, seleccion_optima):
            numero_optimos += 1
    else:
        # ajustar la selección para hacerla factible
        seleccion = factibilizar_seleccion(weights, seleccion, W)
        
        if es_factible(seleccion, weights, W):
            # comparar la f.o entre la selección ajustada y la óptima
            seleccion_optima = etiquetas_test[i][n+1:]
            # ver si es igual a la seleccion optima
            if np.array_equal(seleccion, seleccion_optima):
                numero_optimos_post_factibilizacion += 1

        else:
            print(f"Escenario {i}: No se pudo hacer factible")
            print(f"Selección: {seleccion}")
            print(f"Selección óptima: {seleccion_optima}")


In [ ]:
print(f"Porcentaje de escenarios factibles: {num_factibles / 2000:.2f}")
print(f"Porcentaje de escenarios óptimos en total: {numero_optimos / 2000:.2f}")
print(f"Porcentaje de escenarios óptimos en factibles: {numero_optimos / num_factibles:.2f}")
print(f"Porcentaje de escenarios óptimos post-factibilización: {numero_optimos_post_factibilizacion / (2000 - num_factibles):.2f}")

Porcentaje de escenarios factibles: 0.76
Porcentaje de escenarios óptimos en total: 0.48
Porcentaje de escenarios óptimos en factibles: 0.62
Porcentaje de escenarios óptimos post-factibilización: 0.14
